In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin 

base_url = "https://www.th-bingen.de/" 

def fetch_page(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses (4xx/5xx)
        return BeautifulSoup(response.text, "html.parser")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

def extract_links(soup, base_url):
    links = []
    for a_tag in soup.find_all("a", href=True):  # Find all <a> tags with href attributes
        full_url = urljoin(base_url, a_tag["href"])
        links.append(full_url)  # Add all links without filtering
    return list(set(links))  # Remove duplicates

def crawl_website(start_url, max_depth=2):
    visited = set()  # Track visited URLs to avoid loops
    to_visit = [(start_url, 0)]  # URL and depth
    crawled_data = []  # Store data

    while to_visit:
        current_url, depth = to_visit.pop()
        if current_url in visited or depth > max_depth:
            continue

        visited.add(current_url)
        print(f"Crawling: {current_url} (Depth: {depth})")
        soup = fetch_page(current_url)
        if not soup:
            continue

        # Extract links
        links = extract_links(soup, start_url)
        to_visit.extend((link, depth + 1) for link in links)

        # Extract content
        title = soup.title.string if soup.title else "No title"
        content = " ".join(p.get_text() for p in soup.find_all("p"))
        crawled_data.append({"url": current_url, "title": title, "content": content.strip()})

    return pd.DataFrame(crawled_data)  

# Call the function without relevant_keywords
data = crawl_website(base_url, max_depth=2)
data.head()  # Display the first few rows


Crawling: https://www.th-bingen.de/ (Depth: 0)
Crawling: https://www.th-bingen.de/campus/veranstaltungen-link/veranstaltungsreihen/karrieretag-agrar (Depth: 1)
Crawling: https://www.th-bingen.de/hochschule/presse-oeffentlichkeitsarbeit/mitteilungen (Depth: 2)
Crawling: https://www.th-bingen.de/forschung/forschungsdatenmanagement/veranstaltungsreihe (Depth: 2)
Crawling: https://www.th-bingen.de/campus/beratung/studienerfolgsberatung (Depth: 2)
Crawling: https://www.weincampus-neustadt.de/mba/ (Depth: 2)
Crawling: https://www.th-bingen.de/studiengaenge/klimaschutz-und-anpassung/voraussetzungen (Depth: 2)
Crawling: https://www.th-bingen.de/forschung/hermann-hoepke-institut/schriftenreihe (Depth: 2)
Crawling: https://www.th-bingen.de/einschreibung/fristen-formalitaeten (Depth: 2)
Crawling: https://www.th-bingen.de/hochschule/karriere/stellenangebote (Depth: 2)
Crawling: https://www.th-bingen.de/forschung/standard-titel-1 (Depth: 2)
Crawling: https://www.th-bingen.de/hochschule/th-auf-einen

,url,title,content
0,https://www.th-bingen.de/,Technische Hochschule (TH) Bingen - Willkommen,Seit 1897 bildet die Technische Hochschule Bin...
1,https://www.th-bingen.de/campus/veranstaltunge...,Technischen Hochschule (TH) Bingen: Agrartag,Am Karrieretag Agrar geht es ganz um die beruf...
2,https://www.th-bingen.de/hochschule/presse-oef...,Technischen Hochschule (TH) Bingen: Pressemitt...,Forschungsgelder für smarte Anlagensteuerung i...
3,https://www.th-bingen.de/forschung/forschungsd...,Veranstaltungsreihe,NEUE VERANSTALTUNGSREIHE Die FDM-Landschaft is...
4,https://www.th-bingen.de/campus/beratung/studi...,Technische Hochschule Bingen: Studienerfolgsbe...,Die Studienerfolgsberatung ist eine zentrale E...


In [9]:
data.to_csv("crawled_data_updated .csv", index=False)
